In [1]:
import json
import pandas as pd

#Data Cleaning

### Extract and clean `books.json` data

In [2]:
#import file books.json
with open('books.json', 'r') as file:
    data = json.load(file)

#konversi ke dataframe pandas
df = pd.json_normalize(data['books'])
dfn = pd.DataFrame(df)
dfn

,id,title,author,cover,rating,pages,genre,status
0,1,The Great Gatsby,F. Scott Fitzgerald,https://covers.openlibrary.org/b/id/12749894-L...,4.2,180,Classic,read
1,2,To Kill a Mockingbird,Harper Lee,https://covers.openlibrary.org/b/id/12749895-L...,4.8,376,Fiction,read
2,3,1984,George Orwell,https://covers.openlibrary.org/b/id/12749896-L...,4.6,328,Dystopian,reading
3,4,Pride and Prejudice,Jane Austen,https://covers.openlibrary.org/b/id/12749897-L...,4.4,432,Romance,want-to-read
4,5,The Catcher in the Rye,J.D. Salinger,https://covers.openlibrary.org/b/id/12749898-L...,3.8,277,Coming-of-age,read
5,6,Harry Potter and the Sorcerer's Stone,J.K. Rowling,https://covers.openlibrary.org/b/id/12749899-L...,4.7,309,Fantasy,read
6,7,The Lord of the Rings,J.R.R. Tolkien,https://covers.openlibrary.org/b/id/12749900-L...,4.9,1216,Fantasy,want-to-read
7,8,Dune,Frank Herbert,https://covers.openlibrary.org/b/id/12749901-L...,4.3,688,Sci-Fi,reading


In [3]:
#drop kolom cover dan status
dfn2 = dfn.drop(['cover', 'status'], axis=1)

dfn2

,id,title,author,rating,pages,genre
0,1,The Great Gatsby,F. Scott Fitzgerald,4.2,180,Classic
1,2,To Kill a Mockingbird,Harper Lee,4.8,376,Fiction
2,3,1984,George Orwell,4.6,328,Dystopian
3,4,Pride and Prejudice,Jane Austen,4.4,432,Romance
4,5,The Catcher in the Rye,J.D. Salinger,3.8,277,Coming-of-age
5,6,Harry Potter and the Sorcerer's Stone,J.K. Rowling,4.7,309,Fantasy
6,7,The Lord of the Rings,J.R.R. Tolkien,4.9,1216,Fantasy
7,8,Dune,Frank Herbert,4.3,688,Sci-Fi


In [4]:
#cek missing value
pd.isna(dfn2).sum()

,0
id,0
title,0
author,0
rating,0
pages,0
genre,0


In [5]:
#cek duplikat
dfn2.duplicated().sum()

np.int64(0)

In [6]:
#merubah genre dan status menjadi data category
text_cols = ["title", "author"]
dfn2[text_cols] = dfn2[text_cols].astype("string")

In [7]:
#merubah genre menjadi data category
dfn2["genre"] = dfn2["genre"].astype("category")

In [8]:
#cek tipe data
dfn2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      8 non-null      int64   
 1   title   8 non-null      string  
 2   author  8 non-null      string  
 3   rating  8 non-null      float64 
 4   pages   8 non-null      int64   
 5   genre   8 non-null      category
dtypes: category(1), float64(1), int64(2), string(2)
memory usage: 816.0 bytes


In [9]:
#simpan cleaned data kedalam file csv
dfn2.to_csv("books_clean.csv", index=False)

### Create sample user interaction data and generate reading progress data

In [10]:
import random
from datetime import datetime, timedelta

In [11]:
random.seed(5) #fixed random
users = range(0, 200)
df2 = []

for user in users:
   user = random.randint(1, 100) #random user
   book_id = random.randint(1, 8) #random book
   action = random.choice(["view", "search", "rate"]) #random action
   random_month = random.randint(1, 12) #random month
   timestamp = datetime(2025, random_month, 15, 10, 30) + timedelta(days=random.randint(0, 30), hours=random.randint(0, 23), minutes=random.randint(0, 59)) #random timestamp
   total_pages = dfn.loc[dfn["id"] == book_id, "pages"].values[0] #mengambil total buku
   pages_read = random.randint(1, total_pages) #random halaman buku yang sudah dibaca
   completion_rate = round(pages_read / total_pages, 2) #hitung completion rate
   df2.append({
            "user_id": user,
            "book_id": book_id,
            "action": action,
            "timestamp": timestamp.isoformat(),
            "pages_read": pages_read,
            "total_pages": total_pages,
            "completion_rate": completion_rate
        })
user_df = pd.DataFrame(df2)
user_df

,user_id,book_id,action,timestamp,pages_read,total_pages,completion_rate
0,80,5,rate,2025-07-11T09:23:00,272,277,0.98
1,4,8,view,2025-11-16T15:37:00,381,688,0.55
2,61,4,search,2025-09-19T04:45:00,7,432,0.02
3,94,4,search,2025-05-20T22:40:00,391,432,0.91
4,10,3,rate,2025-10-29T14:38:00,1,328,0.00
...,...,...,...,...,...,...,...
195,69,4,rate,2025-05-06T15:56:00,283,432,0.66
196,33,6,search,2025-10-18T16:46:00,218,309,0.71
197,46,5,search,2025-12-08T06:35:00,60,277,0.22
198,89,2,search,2025-02-19T06:48:00,299,376,0.80


In [12]:
selected_columns = ['user_id', 'book_id', 'action', 'timestamp']
user_interaction_df = user_df[selected_columns]
user_interaction_df

,user_id,book_id,action,timestamp
0,80,5,rate,2025-07-11T09:23:00
1,4,8,view,2025-11-16T15:37:00
2,61,4,search,2025-09-19T04:45:00
3,94,4,search,2025-05-20T22:40:00
4,10,3,rate,2025-10-29T14:38:00
...,...,...,...,...
195,69,4,rate,2025-05-06T15:56:00
196,33,6,search,2025-10-18T16:46:00
197,46,5,search,2025-12-08T06:35:00
198,89,2,search,2025-02-19T06:48:00


In [13]:
user_interaction_df.to_csv("user_interaction.csv", index=False)

In [14]:
selected_columns2 = ['user_id', 'book_id', 'pages_read', 'total_pages',	'completion_rate']
progress_df = user_df[selected_columns2]
progress_df

,user_id,book_id,pages_read,total_pages,completion_rate
0,80,5,272,277,0.98
1,4,8,381,688,0.55
2,61,4,7,432,0.02
3,94,4,391,432,0.91
4,10,3,1,328,0.00
...,...,...,...,...,...
195,69,4,283,432,0.66
196,33,6,218,309,0.71
197,46,5,60,277,0.22
198,89,2,299,376,0.80


In [15]:
progress_df.to_csv("reading_progress.csv", index=False)

# Data Transformation

### Total books read by users

In [16]:
#melihat total buku yang dibaca oleh masing2 pengguna
user_metrics = user_df.groupby("user_id").agg(
    avg_completion=("completion_rate", "mean"),
    total_books=("book_id", "nunique")
).reset_index()

user_metrics

,user_id,avg_completion,total_books
0,1,0.4400,3
1,3,0.7975,4
2,4,0.6000,3
3,5,0.8300,1
4,6,0.6750,2
...,...,...,...
81,94,0.6180,4
82,95,0.1700,1
83,97,0.3650,2
84,98,0.2600,1


### User segment

In [17]:
#melihat segementasi pembaca berdasarkan tiga kategori
user_df["segment"] = pd.cut(
    user_df["completion_rate"],
    bins=[-1, 0.3, 0.7, 1],
    labels=["Low Reader", "Moderate Reader", "High Reader"]
)
user_df

,user_id,book_id,action,timestamp,pages_read,total_pages,completion_rate,segment
0,80,5,rate,2025-07-11T09:23:00,272,277,0.98,High Reader
1,4,8,view,2025-11-16T15:37:00,381,688,0.55,Moderate Reader
2,61,4,search,2025-09-19T04:45:00,7,432,0.02,Low Reader
3,94,4,search,2025-05-20T22:40:00,391,432,0.91,High Reader
4,10,3,rate,2025-10-29T14:38:00,1,328,0.00,Low Reader
...,...,...,...,...,...,...,...,...
195,69,4,rate,2025-05-06T15:56:00,283,432,0.66,Moderate Reader
196,33,6,search,2025-10-18T16:46:00,218,309,0.71,High Reader
197,46,5,search,2025-12-08T06:35:00,60,277,0.22,Low Reader
198,89,2,search,2025-02-19T06:48:00,299,376,0.80,High Reader
